In [2]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [3]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [4]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [18]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
cpi = percentchange(data_source[:CPI], :log) 

real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi

excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)

dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])

term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq

#data = merge(excess_returns, real_t_bill, dy, term, log1p.(data_source[:T_Bill]))
data = merge(excess_returns, real_t_bill, dy, term, cpi)

start = Date(1975, 01,01)
end_d = Date(2020, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq)) 
display(cor(values(s_data)))

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1618,0.7699,0.0713,0.259,-1.0747,-0.7743,-0.0787,0.0036,1.1287,0.4407,0.1145
T_Bill_CPI,0.0166,0.1019,0.0074,0.0291,-0.0832,0.0704,-0.0152,0.3505,0.2942,0.4474,0.0095
Dividend_Stock_Price,0.863,-11.1063,-14.6688,-13.3062,-17.9779,0.2422,-15.943,0.9739,-0.9751,-16.9973,-15.2277
Yeld_10Y_T_Bill,0.0062,0.0397,0.0172,0.0264,-0.0306,-0.7037,0.0084,0.769,0.7198,2.7644,0.019
CPI,0.018,0.1612,0.0351,0.0496,-0.1016,0.3024,0.0159,0.4101,2.0781,1.9539,0.0326


nothing

5×5 Matrix{Float64}:
  1.0         -0.088044   -0.0947785   0.0552798   0.00800211
 -0.088044     1.0         0.183927   -0.0795142  -0.530145
 -0.0947785    0.183927    1.0         0.0762043   0.465928
  0.0552798   -0.0795142   0.0762043   1.0        -0.258195
  0.00800211  -0.530145    0.465928   -0.258195    1.0

In [19]:
assets_names = colnames(s_data)
T, n_assets = size(s_data)
model = VARModel(s_data)
model_summary(model)

,const,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI,std
Total_Stock_T_Bill,0.289,0.019,-2.9824,0.0601,-3.6999,-3.4854,0.079
T_Bill_CPI,-0.0354,-0.0001,0.8692,-0.0071,0.7639,0.741,0.007
Dividend_Stock_Price,-0.2314,-0.0002,1.7257,0.9487,2.751,2.5839,0.0815
Yeld_10Y_T_Bill,0.0016,-0.003,-0.0009,0.0002,0.7731,-0.0014,0.002
CPI,0.0377,0.0049,0.0813,0.0077,-0.6711,0.2179,0.0075


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
Total_Stock_T_Bill,1.0,-0.0,-0.98,0.07,-0.02
T_Bill_CPI,-0.0,1.0,-0.02,-0.12,-0.95
Dividend_Stock_Price,-0.98,-0.02,1.0,-0.07,0.03
Yeld_10Y_T_Bill,0.07,-0.12,-0.07,1.0,-0.12
CPI,-0.02,-0.95,0.03,-0.12,1.0


In [20]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,0.193,-0.0179,-16.7138,0.0097,0.0196
5,0.1034,-0.0137,-16.809,0.0137,0.0152
15,0.0839,-0.0122,-16.9073,0.0149,0.0129
25,0.0796,-0.012,-16.9584,0.0152,0.0121


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,0.1313,0.013,0.295,0.0062,0.0149
5,0.0551,0.0104,0.6417,0.0065,0.0117
15,0.0336,0.0097,0.9081,0.0043,0.0118
25,0.027,0.0096,1.0325,0.0034,0.0124


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,0.0288,0.0097,-0.062,0.0112,-0.0037
5,-0.0014,-0.0318,0.0056,-0.0082,0.0361
15,-0.0379,-0.0325,0.0321,0.0015,0.0305
25,0.0273,-0.0058,0.0167,0.0383,0.0178


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,0.0266,0.0291,-0.0123,0.0398,-0.0833
5,-0.0088,0.0028,0.0015,-0.0422,0.0587
15,-0.0839,-0.0612,0.0737,0.0164,0.0625
25,0.0564,-0.0385,0.016,0.0598,0.04


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.109,-0.0636,-0.0238,0.1049,0.1915,0.2804,0.4092,0.4531,0.5019
5,-0.0226,-0.0039,0.0124,0.0665,0.1032,0.141,0.1938,0.2117,0.2307
15,0.0048,0.0171,0.0282,0.0612,0.0845,0.1063,0.1385,0.1489,0.1612
25,0.017,0.0273,0.0352,0.0617,0.0794,0.0978,0.1235,0.1321,0.1434


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0485,-0.0434,-0.039,-0.0267,-0.018,-0.009,0.0036,0.0075,0.0126
5,-0.0381,-0.034,-0.0311,-0.0207,-0.0137,-0.0067,0.0034,0.0068,0.0102
15,-0.035,-0.031,-0.028,-0.0189,-0.0121,-0.0055,0.0039,0.0069,0.0098
25,-0.0345,-0.0308,-0.0277,-0.0185,-0.0121,-0.0055,0.0038,0.0068,0.0098


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-17.4082,-17.2983,-17.1998,-16.9131,-16.7069,-16.5145,-16.2357,-16.1353,-16.0469
5,-18.3096,-18.0765,-17.8589,-17.2488,-16.8132,-16.3809,-15.7462,-15.5316,-15.308
15,-19.047,-18.6726,-18.3816,-17.5193,-16.9117,-16.3105,-15.3981,-15.0941,-14.7674
25,-19.3747,-18.9683,-18.637,-17.6392,-16.9708,-16.2668,-15.2453,-14.9118,-14.5646


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0047,-0.0025,-0.0007,0.0056,0.0097,0.0139,0.0199,0.022,0.0243
5,-0.0015,0.0007,0.003,0.0093,0.0137,0.0181,0.0242,0.0263,0.0286
15,0.0051,0.0065,0.008,0.012,0.0149,0.0178,0.022,0.0234,0.0249
25,0.0073,0.0086,0.0097,0.0129,0.0151,0.0174,0.0208,0.0219,0.0231


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0143,-0.0099,-0.0051,0.0094,0.0196,0.0298,0.0441,0.0483,0.0542
5,-0.0114,-0.0074,-0.0039,0.0074,0.0152,0.0228,0.0347,0.0386,0.0427
15,-0.0146,-0.0097,-0.0062,0.0051,0.0129,0.0209,0.0323,0.0365,0.041
25,-0.0168,-0.0123,-0.0079,0.0038,0.0122,0.0204,0.0326,0.0368,0.0409


In [21]:
real_equity_scenarios = reshape(scenarios[1, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)
real_bond_scenarios = reshape(scenarios[2, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)

ret_in_years = cum_returns_in_periods(vcat(real_equity_scenarios, real_bond_scenarios), periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names[1:2], string.(periods))

for a in 1:2
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  



,Total_Stock_T_Bill,T_Bill_CPI
1,-16.5207,-16.7317
5,-16.7056,-16.8228
15,-16.8235,-16.9195
25,-16.8788,-16.9704


,Total_Stock_T_Bill,T_Bill_CPI
1,0.1875,0.2936
5,0.6032,0.6396
15,0.8851,0.9114
25,1.0127,1.0382


,Total_Stock_T_Bill,T_Bill_CPI
1,-0.0626,-0.0615
5,0.0078,0.0045
15,0.0314,0.0319
25,0.0179,0.0165


,Total_Stock_T_Bill,T_Bill_CPI
1,-0.0351,-0.0125
5,-0.0097,0.0005
15,0.0782,0.0715
25,0.0156,0.0151


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-16.9594,-16.8859,-16.8336,-16.6478,-16.517,-16.3934,-16.2179,-16.1596,-16.0943
5,-18.1071,-17.8861,-17.6946,-17.1219,-16.7114,-16.2977,-15.7079,-15.5057,-15.3036
15,-18.9123,-18.5429,-18.2651,-17.4216,-16.826,-16.2405,-15.349,-15.0579,-14.7353
25,-19.2551,-18.8466,-18.5305,-17.5485,-16.8917,-16.1988,-15.1987,-14.8674,-14.5354


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-17.4261,-17.3124,-17.2138,-16.9294,-16.7242,-16.534,-16.2549,-16.1603,-16.0626
5,-18.3183,-18.0849,-17.8692,-17.2599,-16.8277,-16.3947,-15.7631,-15.5471,-15.3234
15,-19.0655,-18.6818,-18.4015,-17.5339,-16.9238,-16.3184,-15.4088,-15.0983,-14.7701
25,-19.3947,-18.9942,-18.6617,-17.6541,-16.9828,-16.2752,-15.2479,-14.9177,-14.562


In [141]:
mdd, mddl = max_drawdown_and_length(real_equity_scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(real_bond_scenarios[1,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.15603  0.219442  0.268403  0.329568  0.451828

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 17.0  29.0  41.0  57.0  88.0

:Price_10Y_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.124018  0.174967  0.212861  0.25475  0.340629

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 28.0  46.0  61.0  79.0  98.0